In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

Возьмем данные о сделках компании Boeing на поставку самолетов в различные страны в период с 1958 по 2022 года. В качестве клиентов у компании предлагается рассмотреть не компании, а целые страны (для "укрупнения" метрик выборки). Никакой дополнительной обработки данных (по типу фильтрации выбросов) производить не будем (посчитал что так будет интереснее)

In [ ]:
df = pd.read_csv('/content/cleanedOrdersBoeing (1).csv')

df.head()

,Unnamed: 0,country,customer,delivery_year,engine,model,month,year,region,delivery_total,order_total,unfilled_orders
0,0,Afghanistan,Ariana Afghan Airlines,1968,PW,727,Mar,1968,Central Asia,1,1,0
1,1,Afghanistan,Ariana Afghan Airlines,1970,PW,727,Apr,1969,Central Asia,1,1,0
2,2,Afghanistan,Ariana Afghan Airlines,1979,GE,DC-10,Sep,1978,Central Asia,1,1,0
3,4,Algeria,Air Algerie,1974,PW,727,Jan,1974,Africa,1,1,0
4,5,Algeria,Air Algerie,1974,PW,737-200,Jan,1974,Africa,1,1,0


Поля датасета:



*   ***country*** - Страна покупателя самолетов
*   ***customer*** - Клиент-авиакомпания
*   ***delivery_year*** - год доставки самолета
*   ***engine*** - мотор, установленный в самолете
*   ***model*** - модель самолета
*   ***month*** - месяц поставки
*   ***year*** - год заключения сделки
*   ***delivery_total*** - общее количество доставленных самолетов
*   ***order_total*** - общее количество заказанных самолетов
*   ***unfilled_orders*** - число недоставленных по изначальному заказу самолетов



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8002 entries, 0 to 8001
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       8002 non-null   int64 
 1   country          8002 non-null   object
 2   customer         8002 non-null   object
 3   delivery_year    8002 non-null   int64 
 4   engine           8002 non-null   object
 5   model            8002 non-null   object
 6   month            8002 non-null   object
 7   year             8002 non-null   int64 
 8   region           8002 non-null   object
 9   delivery_total   8002 non-null   int64 
 10  order_total      8002 non-null   int64 
 11  unfilled_orders  8002 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 750.3+ KB


Итак, создатели датасета, изначально выделили три сильно связанных метрики: *количество заказанных, количество доставленных и количество недоставленных самолетов*. Однако сперва стоит проверить информативность третьей метрики (вдруг она почти всюду равна нулю)

Для этого проверим при помощи одновыборочного критерия Стьюдента *равенство среднего значения 0 против двухсторонней альтернативы* (здесь и далее уровень значимости проверяемых гипотез - 0.05)

In [ ]:
st.ttest_1samp(df.groupby('country')['unfilled_orders'].sum(), popmean=0)

TtestResult(statistic=-1.8724142027918698, pvalue=0.06346717297612614, df=126)

Полученный p-уровень больше заданного, следовательное *нулевую гипотезу мы не отвергаем и можем сказать, что данная метрика не особо информативна*.

Предлагается ввести другую метрику - ***задержка доставки*** (***delivery_lag***); численно она будет равна разности между годом оформления заказа и годом его получения

In [ ]:
df['delivery_lag'] = df['delivery_year'] - df['year']

Введем в рассмотрение три временных сегмента заказов:



1.   ***Ранние*** - заказы совершенные в период с **1958** по **1980** годы
2.   ***Срединные*** - заказы совершенные в период с **1981** по **2000** годы
3.   ***Позднейшие*** - заказы совершенные в период с **2001** по **2022** годы



In [ ]:
early_years_df = (
    df
    .loc[df['delivery_year'] <= 1980]
    .groupby('country')['delivery_total', 'order_total', 'delivery_lag']
    .sum()
    .reset_index()
)

middle_years_df = (
    df
    .loc[(1980 < df['delivery_year']) & (2000 >= df['delivery_year'])]
    .groupby('country')['delivery_total', 'order_total', 'delivery_lag']
    .sum()
    .reset_index()
)

late_years_df = (
    df
    .loc[2000 < df['delivery_year']]
    .groupby('country')['delivery_total', 'order_total', 'delivery_lag']
    .sum()
    .reset_index()
)

# **Исследование гипотез о среднем. Критерии Стьюдента и Фишера**

Сперва поработаем с метрикой общего числа поставок. Мы хотим посмотреть, **увеличилось ли в среднем число поставляемых самолетов через периоды**. Для этого воспользуемся *двухвыборочным критерием Стьюдента с одностороннней альтернативой* (в более раннем периоде в среднем было заказов **меньше**).

In [ ]:
st.ttest_ind(early_years_df['delivery_total'], middle_years_df['delivery_total'], alternative='less')

TtestResult(statistic=-0.4140212821228058, pvalue=0.3396716792448685, df=183.0)

In [ ]:
st.ttest_ind(middle_years_df['delivery_total'], late_years_df['delivery_total'], alternative='less')

TtestResult(statistic=-0.6508546117577634, pvalue=0.2579969999033299, df=175.0)

In [ ]:
st.ttest_ind(early_years_df['delivery_total'], late_years_df['delivery_total'], alternative='less')

TtestResult(statistic=-1.0538279347257238, pvalue=0.14671991337617413, df=172.0)

Как можно заметить, во всех трех парах p-уровень получился *больше заданного уровня значимости*, а значит мы *не можем утверждать, что число поставок в среднем увеличилось со временем*

Убедимся, что и *двухвыборочный Критерий Фишера* даст аналогичные результаты

In [ ]:
st.f_oneway(early_years_df['delivery_total'], middle_years_df['delivery_total'])

F_onewayResult(statistic=0.17141362205061195, pvalue=0.6793433584897393)

In [ ]:
st.f_oneway(middle_years_df['delivery_total'], late_years_df['delivery_total'])

F_onewayResult(statistic=0.42361172564634875, pvalue=0.5159939998066674)

In [ ]:
st.f_oneway(early_years_df['delivery_total'], late_years_df['delivery_total'])

F_onewayResult(statistic=1.1105533160082843, pvalue=0.2934398267523411)

Наконец, проверим одновременное равенство средних всех трех периодов с помощью того же *критерия Фишера*, но уже для несколько сегментов

In [ ]:
st.f_oneway(early_years_df['delivery_total'], middle_years_df['delivery_total'], late_years_df['delivery_total'])

F_onewayResult(statistic=0.5768117503540394, pvalue=0.5623899176306286)

P-уровень получился больше заданного уровня значимости, следовательно **гипотезу о равенстве всех средних мы не отвергаем**

Теперь рассмотрим введенную выше метрику задержки поставки. Мы хотим узнать, **изменялась ли средняя задержка от одного периода к другому**. Для этого будем проверять равенство средних значений по сегментам, используя *двухвыборочный критерий Стьюдента с двухсторонней альтернативой*

In [ ]:
st.ttest_ind(early_years_df['delivery_lag'], middle_years_df['delivery_lag'])

TtestResult(statistic=-1.2271560224224747, pvalue=0.22134030071743296, df=183.0)

In [ ]:
st.ttest_ind(middle_years_df['delivery_lag'], late_years_df['delivery_lag'])

TtestResult(statistic=-1.4182684916756096, pvalue=0.15789070578750794, df=175.0)

In [ ]:
st.ttest_ind(early_years_df['delivery_lag'], late_years_df['delivery_lag'])

TtestResult(statistic=-2.139064500744008, pvalue=0.03384108147505869, df=172.0)

Для первых двух пар мы *не отвергаем гипотезу о равенстве средних* (так как p-уровень больше заданного уровня значимости). В последнем же случае *гипотеза о равенстве средних отвергается*, а значит *среднее время задержки отличается для поставок из раннего временного сегмента и позднего*. Чтобы понять, в какую сторону изменилось среднее значение задержки, можем вновь применить *двухвыборочный критерий Стьюдента*, но уже *с левосторонней альтернативой*.

In [ ]:
st.ttest_ind(early_years_df['delivery_lag'], late_years_df['delivery_lag'], alternative='less')

TtestResult(statistic=-2.139064500744008, pvalue=0.016920540737529344, df=172.0)

Нулевая гипотеза о равенстве средних вновь *отвергнута в пользу односторонней альтернативы*, а значит мы можем утверждать, что *средняя задержка для доставки увеличилась с течением времени*. Это можно попробовать обосновать как усложнившимися технологиями производства, так и возросшим доверием авиакомпаний к производителю (то есть их готовностью заключать контракты с более длительным сроком реализации, не боясь, что компания-поставщик разорится или не сможет выполнить контрактыне обязательства)

Проделаем те же самые проверки, используя *критерий Фишера*

In [ ]:
st.f_oneway(early_years_df['delivery_lag'], middle_years_df['delivery_lag'])

F_onewayResult(statistic=1.5059119033677502, pvalue=0.22134030071743344)

In [ ]:
st.f_oneway(middle_years_df['delivery_lag'], late_years_df['delivery_lag'])

F_onewayResult(statistic=2.011485514479808, pvalue=0.15789070578750844)

In [ ]:
st.f_oneway(early_years_df['delivery_lag'], late_years_df['delivery_lag'])

F_onewayResult(statistic=4.575596938343213, pvalue=0.03384108147505868)

Используя *двухвыборочный критерий Фишера* мы получили аналогичные результаты: в первых двух случаях *нулевую гипотезу о равенстве средних не отвергаем*, в последнем - *отвергаем*

Попробуем применить *критерий Фишера* сразу для всех трех сегментов

In [ ]:
st.f_oneway(early_years_df['delivery_lag'], middle_years_df['delivery_lag'], late_years_df['delivery_lag'])

F_onewayResult(statistic=2.989890692062178, pvalue=0.05199249129603271)

На удивление, гипотезу о равенстве средних на текущем уровне значимости *не отвергаем*. На самом деле, применение критерия Фишера в нашем случае несовсем правомерно, ибо *дисперсии величин сильно различаются* (сейчас достаточно будет поглядеть на разницу в выборочных дисперсиях, формально проверим это ниже)

In [ ]:
early_years_df['delivery_lag'].std(ddof=1)

132.87680634875926

In [ ]:
middle_years_df['delivery_lag'].std(ddof=1)

236.67476238082727

In [ ]:
late_years_df['delivery_lag'].std(ddof=1)

503.6706867989471

**Доверительные интервалы для средних**

Попробуем построить доверительные интервалы для среднего исследуемых метрик по сегментам

In [ ]:
def precise_interval(rvs, alpha=0.05):
  """
      Построение точного доверительного интервала для среднего значения по выборке rvs на уровне доверия 1 - alpha
  """
  n = rvs.shape[0]
  t_crit = st.t.ppf(1 - alpha / 2, df=n-1, loc=0, scale=1) # Получение необходимой квантили стандартного нормального распределения

  left = rvs.mean() - t_crit * np.sqrt(rvs.var(ddof=1) / n) # Расчет левой границы интервала
  right = rvs.mean() + t_crit * np.sqrt(rvs.var(ddof=1) / n) # Расчет правой границы интервала

  return left, right

In [ ]:
def bootstrap_interval(rvs, stat=np.mean, alpha=0.05):
  """
      Построение эфронова доверительного интервала для выбранной статистики stat (передаваемой в виде функции) по выборке rvs на уровне доверия 1 - alpha
  """
  n = rvs.shape[0]
  distribution_lst = []
  for _ in range(100):
    a_ = np.random.choice(rvs, size=n, replace=True) # Получение подвыборки
    distribution_lst.append(stat(a_)) # Расчет искомой статистики для подвыборки и добавление в псевдораспределение

  return np.percentile(distribution_lst, q=[alpha / 2 * 100, (1 - alpha / 2) * 100]) # Возвращение требуемого доверительного интервала по полученному псевдораспределению статистики

Сперва построим точные доверительные интервалы

In [ ]:
for metrics in ['delivery_total', 'delivery_lag']:
  print("Точные доверительные интервалы для " + metrics)
  print(f"early_years_df - {precise_interval(early_years_df[metrics])}")
  print(f"middle_years_df - {precise_interval(middle_years_df[metrics])}")
  print(f"late_years_df - {precise_interval(late_years_df[metrics])}")
  print("---")

Точные доверительные интервалы для delivery_total
early_years_df - (-6.485055463704434, 130.44109941974838)
middle_years_df - (4.474575964609343, 163.31265807794387)
late_years_df - (25.22188185935441, 225.35643139365766)
---
Точные доверительные интервалы для delivery_lag
early_years_df - (4.425966826533969, 59.77183537126824)
middle_years_df - (18.40729691847578, 115.35866052833272)
late_years_df - (39.261407434752684, 259.22052027609067)
---


В случае для delivery_total мы получили интервалы с достаточно большим общим пересечением, что может косвенно свидетельствовать в пользу совпадения средних (для первого сегмента левая граница оказалась меньше нуля, что не моежт быть физически, поэтому можно считать, что там стоит 0)

Для delivery_lag получившиеся интервалы также имеют общее пересечение, однако в случае первого и третьего сегмента это пересечение уже достаточно мало (следовательно велик шанс, что среднее у них не сопадет, что мы и получили выше)

Теперь для построения доверительных интервалов воспользуемся бутстрэпом

In [ ]:
for metrics in ['delivery_total', 'delivery_lag']:
  print("Эфроновы доверительные интервалы для " + metrics)
  print(f"early_years_df - {bootstrap_interval(early_years_df[metrics])}")
  print(f"middle_years_df - {bootstrap_interval(middle_years_df[metrics])}")
  print(f"late_years_df - {bootstrap_interval(late_years_df[metrics])}")
  print("---")

Эфроновы доверительные интервалы для delivery_total
early_years_df - [ 21.19752747 126.74065934]
middle_years_df - [ 33.73510638 165.15106383]
late_years_df - [ 56.87018072 233.30150602]
---
Эфроновы доверительные интервалы для delivery_lag
early_years_df - [14.77142857 60.73214286]
middle_years_df - [ 34.35691489 114.06861702]
late_years_df - [ 74.95722892 263.91415663]
---


По полученным эфроновым интервалам можно во многом сделать те же выводы, что и для точнх; примечательно лишь то, что для delivery_lag доверительный интервал для среднего у первого сегмента вообще не имеет пересечения с интервалом у третьего, что опять же сигнализирует и возможном расхождении средних значений у этих сегментов

# **Cхожесть функций распределения. Критерий Манна-Уитни**

Рассмотрим ранее неиспользовавшуюся метрику order_total. В предыдущем разделе мы увидели, что непоставки самолета начали происходить только в последнем периоде, а значит можно предположить, что большая часть заказов исполнена полностью. Давайте проверим **схожесть распределений метрик delivery_total и order_total** по всему датасету при помощи *критерия Манна-Уитни с двухсторонней альтернативой*

In [ ]:
st.mannwhitneyu(df.groupby('country')['delivery_total'].sum(), df.groupby('country')['order_total'].sum())

MannwhitneyuResult(statistic=8066.5, pvalue=0.9979528673615058)

Как мы видим, p-уровень явно больше заданного уровня значимости, а значит нулевую *гипотезу о схожести распределений мы не отвергаем*

Применим тот же критерий для каждого сегмента

In [ ]:
st.mannwhitneyu(early_years_df['delivery_total'].values, early_years_df['order_total'].values)

MannwhitneyuResult(statistic=4140.5, pvalue=1.0)

In [ ]:
st.mannwhitneyu(middle_years_df['delivery_total'].values, middle_years_df['order_total'].values)

MannwhitneyuResult(statistic=4418.0, pvalue=1.0)

In [ ]:
st.mannwhitneyu(late_years_df['delivery_total'].values, late_years_df['order_total'].values)

MannwhitneyuResult(statistic=3448.0, pvalue=0.9922634986814639)

И снова во всех случаях мы не можем отвергнуть нулевую гипотезу, а значит *распределение этих метрик совпадает и внутри различных периодов*; другими словами, статистически значимым будет вывод о *полном исполнении своих обязательств компанией*

Теперь давайте применим *критерий Манна-Уитни* для различных сегментов чтобы понять, **изменялось ли распределение объемов заказов** (то есть метрики order_total) **с течением времени**

In [ ]:
st.mannwhitneyu(early_years_df['order_total'], middle_years_df['order_total'])

MannwhitneyuResult(statistic=4205.5, pvalue=0.8451584551126978)

In [ ]:
st.mannwhitneyu(middle_years_df['order_total'], late_years_df['order_total'])

MannwhitneyuResult(statistic=3279.5, pvalue=0.06761065903819423)

In [ ]:
st.mannwhitneyu(early_years_df['order_total'], late_years_df['order_total'])

MannwhitneyuResult(statistic=3062.5, pvalue=0.031417456950211184)

По полученным результатам можно сделать вывод, что р*аспределение объемов заказов в соседних периодах не имеет статистически значимых различий*, тогда как между первым и третьим периодами таковые имеются

Наконец, сравним распределения задержек перед поставками для всех трех периодов (опять используя *критерий Манна-Уитни*)

In [ ]:
st.mannwhitneyu(early_years_df['delivery_lag'], middle_years_df['delivery_lag'])

MannwhitneyuResult(statistic=3899.0, pvalue=0.29877874165598317)

In [ ]:
st.mannwhitneyu(middle_years_df['delivery_lag'], late_years_df['delivery_lag'])

MannwhitneyuResult(statistic=2572.0, pvalue=9.275965599645297e-05)

In [ ]:
st.mannwhitneyu(early_years_df['delivery_lag'], late_years_df['delivery_lag'])

MannwhitneyuResult(statistic=1982.5, pvalue=6.31812417854995e-08)

Получили, что для поставок в период с 1958 по 1980 и 1981 по 2000 *задержки поставок имеют достаточно схожее распределение*, тогда как в периоде с 2001 по 2022 *распределение задержки сильно отличается от остальных*

# **Примеры других статистических гипотез**

Конечно, наиболее адекватным и правильным было бы **проверить распределения метрик на нормальность**. Проделаем это по всему датасету, используя *критерий Шапиро-Уилка*

In [ ]:
st.shapiro(df.groupby('country')['delivery_total'].sum())

ShapiroResult(statistic=0.15802907943725586, pvalue=8.980480108596478e-24)

In [ ]:
st.shapiro(df.groupby('country')['order_total'].sum())

ShapiroResult(statistic=0.15798640251159668, pvalue=8.970124731493642e-24)

In [ ]:
st.shapiro(df.groupby('country')['delivery_lag'].sum())

ShapiroResult(statistic=0.20227235555648804, pvalue=3.028663260451082e-23)

Как несложно понять, все эти метрики имеют **распределения крайне далекие от нормального**; это ставит под *глубокое сомнение релевантность применения всех использованных ранее тестов Стьюдента и Фишера*.

Еще было бы неплохо проверить попарное совпадение дисперсий различных сегментов для тех метрик, где мы применяли тест ANOVA (delivery_lag и unfilled_orders). Для этого воспользуемся F-критерием из статистики (по правде говоря, для него тоже желательно иметь распределения, не сильно отличающиеся от нормального)

In [ ]:
def f_test(x, y):
 """
    Проверяет гипотезу о равенстве дисперсий двух распределений против двухсторонней альтернативы
 """
 f = x.std(ddof=1) / y.std(ddof=1) # вычисляем наблюдаемую статистику
 dfn = x.size-1 # находим первое число степеней свободы
 dfd = y.size-1 # находим второе число степеней свободы
 p = st.f.cdf(f, dfn, dfd) # находим p-уровень для наблюдаемой статистики
 return p

Проверим равенство дисперсии для сегментов по метрике **delivery_total**

In [ ]:
f_test(early_years_df['delivery_total'], middle_years_df['delivery_total'])

0.21609899295835822

In [ ]:
f_test(middle_years_df['delivery_total'], late_years_df['delivery_total'])

0.21675228160327992

In [ ]:
f_test(early_years_df['delivery_total'], late_years_df['delivery_total'])

0.06201968172199329

Как можно увидеть, во всех случаях можно утверждать, что *статистически значимых различий в дисперсиях нет*, а значит *применение ANOVA к данной метрике возможно* (если не учитывать отсутствие нормальности распределения)

Теперь проверим то же самое для метрики delivery_lag

In [ ]:
f_test(early_years_df['delivery_lag'], middle_years_df['delivery_lag'])

0.003176764669812119

In [ ]:
f_test(middle_years_df['delivery_lag'], late_years_df['delivery_lag'])

0.00022718160366851128

In [ ]:
f_test(early_years_df['delivery_lag'], late_years_df['delivery_lag'])

9.434077812035592e-10

В этот же раз во всех случаях мы *получили статистически значимые различия в дисперсиях*, следовательно здесь *применение ANOVA невозможно* (что отразилось в парадоксальном выводе в соответствующем разделе выше)